In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import *
from pyspark import SparkConf
import pyspark as ps
import pandas as pd

In [2]:
conf = (SparkConf()
            .setAppName("TestRdd") \
            .set('spark.driver.cores', '1') \
            .set('spark.executor.cores', '3') \
            .set('spark.driver.memory', '4G') \
            .set('spark.executor.memory', '16G'))\
            .set("spark.network.timeout", "15000s")
sc = SparkContext('local[4]', conf = conf)
sqlContext = SQLContext(sc)

In [3]:
data_negocios = sqlContext.read.csv('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/business.csv', sep=',', multiLine=True,header=True, inferSchema=True)
data_reviews =  sqlContext.read.parquet('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/review.parquet')

In [4]:
negocios = data_negocios.rdd
reviews = data_reviews.rdd

In [5]:
negocios_california = negocios.filter(lambda x: x.state == "CA").map(lambda x: (x.business_id,x.state))
#Obtengo los negocios del estado de california
#Mapeo (negocio_id,estado)

In [12]:
reviews_dict = reviews.map(lambda x: (x.business_id,x.user_id))
#Mapeo (negocio_id,usuario_id)

In [13]:
review_negocios = negocios_california.join(reviews_dict)

In [18]:
usuario = review_negocios.map(lambda x: (x[1][1],1))\
                         .reduceByKey(lambda x,y: x + y)\
                         .reduce(lambda x,y: x if x[1] > y[1] else y)

In [16]:
data_usuarios =  sqlContext.read.csv('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/user.csv', sep=',', multiLine=True,header=True, inferSchema=True)

In [22]:
usuarios = data_usuarios.rdd
nombre = usuarios.filter(lambda x: x.user_id == usuario[0]).map(lambda x: x.name).first()

In [25]:
print("El usuario que hizo mas reviews en California es", nombre, "con",usuario[1],"reviews")

El usuario que hizo mas reviews en California es Eric con 447 reviews
